In [2]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import numpy as np

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import helper
from dataset import TimeSeriesDataset

## Training

In [3]:
from Generator_transformer import Generator
from Discriminator_transformer import Discriminator
from F1_score_check import F1_score_check
from GAN import GAN

from DeepConvLSTM_model import DeepConvNet
val_model = DeepConvNet(hidden_size = 256)
state_dict = torch.load("val_model.ckpt")["state_dict"]

# from PAMAP2_model_acc import DeepConvNet
# val_model = DeepConvNet(hidden_size = 512)
# state_dict = torch.load("acc_transformer.ckpt")["state_dict"]

print("Loading Validation model")
state_dict = helper.remove_prefix_from_dict("model.", state_dict)
val_model.load_state_dict(state_dict, strict = False)
val_model.eval()

start_activity = 1
total_activities = 7
val_iter_size = 3 # Num of validation iterations to perform
batch_size = 32
data_size = (27, 100)
noise_len = 100
period = 100
max_retries = 2
init_dim_feedforward = 2048
dim_feedforward_exponent = 5

success = {}

for chosen_activity in range(start_activity, total_activities+1):
    try_num = 0
    dim_feedforward = init_dim_feedforward
    data = helper.load_PAMAP2_activity(activity_num = chosen_activity)
    dtset = TimeSeriesDataset(data)
    train_iter = torch.utils.data.DataLoader(dtset, batch_size = batch_size, shuffle = True, num_workers = 10, pin_memory = True)
    val = torch.ones((batch_size * val_iter_size, 1))
    val_iter = torch.utils.data.DataLoader(val, batch_size = batch_size, num_workers = 10, pin_memory = True)

    while (try_num < max_retries):
        print("Activity ", chosen_activity,", Try ",try_num)
        model = GAN(val_model = val_model, 
                    generator = Generator(noise_len = noise_len, output_size = data_size, nheads = 5, period = period, dim_feedforward = dim_feedforward),
                    discriminator = Discriminator(input_size = data_size, nheads = 5, period = period, dim_feedforward = dim_feedforward),
                    val_expected_output = chosen_activity - 1,
                    noise_len = noise_len,
                    decay = 0,
                    dis_lr = 0.0002,
                    gen_lr = 0.0002,
                   )

        trainer = pl.Trainer(gpus=-1,
                             max_epochs=100,
                             callbacks = [F1_score_check(threshold_value = 0.95), 
                                         ], # Early stopping callback
                             logger = TensorBoardLogger(save_dir = 'Transformer_GAN_logs/', name = "PAMAP2_act_"+str(chosen_activity)),
                             check_val_every_n_epoch = 5,
                             )
        result = trainer.fit(model, train_iter, val_iter)
        # verify if the model is trained
        if trainer.callback_metrics['val_f1_score'] >=0.95 or result != 1:
            print("Success!")
            success[chosen_activity] = trainer.logger.version
            break
        else: # model not trained
            dim_feedforward *= dim_feedforward_exponent
            try_num += 1
            if try_num == max_retries:
                success[chosen_activity] = None
                
print(success)

Loading Validation model
File exists. Loading
Keep only activity number  1
Windowing


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Done!
Activity  1 , Try  0



  | Name          | Type          | Params
------------------------------------------------
0 | criterion     | BCELoss       | 0     
1 | generator     | Generator     | 1 M   
2 | discriminator | Discriminator | 7 M   
3 | val_model     | DeepConvNet   | 3 M   



Success!
File exists. Loading
Keep only activity number  2
Windowing


...

Done!
Activity  7 , Try  0



  | Name          | Type          | Params
------------------------------------------------
0 | criterion     | BCELoss       | 0     
1 | generator     | Generator     | 1 M   
2 | discriminator | Discriminator | 7 M   
3 | val_model     | DeepConvNet   | 3 M   



Activity  7 , Try  1


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params
------------------------------------------------
0 | criterion     | BCELoss       | 0     
1 | generator     | Generator     | 2 M   
2 | discriminator | Discriminator | 8 M   
3 | val_model     | DeepConvNet   | 3 M   



Success!
{1: 19, 2: 8, 3: 15, 4: 8, 5: None, 6: 4, 7: 11}
